In [ ]:
  !pip install torch==1.9.0+cu102 torchtext==0.10.0 torchvision==0.10.0+cu102 torch-optimizer==0.1.0 -f https://download.pytorch.org/whl/torch/ -f https://download.pytorch.org/whl/torchvision/
  !git clone https://github.com/openai/CLIP
  # !pip install taming-transformers
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf clipit
  !git clone https://github.com/mfrashad/clipit.git
  !pip install ftfy regex tqdm omegaconf pytorch-lightning
  !pip install kornia==0.6.1
  !pip install imageio-ffmpeg   
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  # ClipDraw deps
  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom

  !pip install fastapi
  !pip install python-multipart>=0.0.5
  !pip install pyngrok uvicorn
  !pip install requests
  !pip install orjson

  !pip3 install opencv-contrib-python
  
  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  # !ls
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  
  !mkdir -p steps
  !mkdir -p models


In [ ]:
import sys
import IPython
import os


sys.path.append("clipit")
result_msg = "setup complete"
if not os.path.isfile("first_init_complete"):
  # put stuff in here that should only happen once
  !wget https://github.com/fannymonori/TF-LapSRN/raw/master/export/LapSRN_x4.pb
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu, and then run Setup again"


In [ ]:
import clipit
import torch
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, File, UploadFile, Form, BackgroundTasks
from fastapi.responses import ORJSONResponse
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import cv2
import requests
import base64

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}

class GenerateReqBody(BaseModel):
  prompts: str = "Underwater World"

@app.post("/generate", response_class=ORJSONResponse)
async def generate(
        reqBody: GenerateReqBody
    ):
    print(reqBody.prompts)
    torch.cuda.empty_cache()
    clipit.reset_settings()
    reqBody.prompts = reqBody.prompts

    clipit.add_settings(prompts=reqBody.prompts,
                        iterations=110,
                        aspect="square",
                        quality="draft",
                        scale=1.5,
                        use_pixeldraw=False,
                        use_clipdraw=False,
                        make_video=False)
    
    settings = clipit.apply_settings()
    clipit.do_init(settings)
    clipit.do_run(settings)

    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "LapSRN_x4.pb"
    sr.readModel(path)
    sr.setModel("lapsrn", 4)
    sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    img = cv2.imread('output.png')
    result = sr.upsample(img)
    cv2.imwrite('output_inc.png', result)
    
    with open("output_inc.png", "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
    return {"image_string": my_string}
  

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print('Doc URL:', ngrok_tunnel.public_url+'/docs')
data = {
    "updateObj": {
        "aiUrl": ngrok_tunnel.public_url
    }
}
print(data)
r = requests.post("https://defining-art.herokuapp.com/util/updateMetaData", json=data)
print(r)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

In [ ]:
!pip3 install opencv-contrib-python

In [3]:
!ngrok authtoken 23bi7CDQIj6Kn6fDyr6vrIaKbD6_51YQ67xd6CgBaFqUpXYG4

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
